<a href="https://colab.research.google.com/github/gwisnu/mapping-pemilu2019/blob/master/ts_mapping_pemilu2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import folium
import json 
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time

In [0]:
timestamp = int(time.time())
#print(timestamp)

In [7]:
url_kawalpemilu ='https://kawal-c1.appspot.com/api/c/0?'+str(timestamp)
url_kpu = 'https://pemilu2019.kpu.go.id/static/json/hhcw/ppwp.json'
response = requests.get(url_kawalpemilu ,verify=False)
data = json.loads(response.text)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [36]:
print(data)

{'id': 0, 'name': 'IDN', 'depth': 0, 'parentIds': [], 'parentNames': [], 'children': [[1, 'ACEH', 17389, 1734674, 1789100], [6728, 'SUMATERA UTARA', 42668, 4838923, 4946830], [12920, 'SUMATERA BARAT', 16719, 1836875, 1881128], [14086, 'RIAU', 17642, 1963184, 1900013], [15885, 'JAMBI', 11342, 1249398, 1226257], [17404, 'SUMATERA SELATAN', 25332, 2965679, 2911896], [20802, 'BENGKULU', 6165, 708242, 690866], [22328, 'LAMPUNG', 28060, 3101750, 2972387], [24993, 'KEPULAUAN BANGKA BELITUNG', 3803, 475784, 456785], [25405, 'KEPULAUAN RIAU', 5478, 619882, 609542], [25823, 'DKI JAKARTA', 29063, 3874021, 3887577], [26141, 'JAWA BARAT', 138136, 16724460, 16546385], [32676, 'JAWA TENGAH', 115407, 13901364, 13995538], [41863, 'DAERAH ISTIMEWA YOGYAKARTA', 11781, 1330110, 1401764], [42385, 'JAWA TIMUR', 130184, 15226058, 15686936], [51578, 'BANTEN', 33471, 4100146, 4012331], [53241, 'BALI', 12386, 1556427, 1573861], [54020, 'NUSA TENGGARA BARAT', 15989, 1800915, 1866338], [55065, 'NUSA TENGGARA TIMU

In [0]:
table = list()
row ={}
for i in (data['data']):
    flat_json = json_normalize(data['data'][i],errors='ignore')
    row.update(row)
    table.append({"code_adm":i,"pas01":(flat_json.iloc[0]['sum.pas1']),"pas02":(flat_json.iloc[0]['sum.pas2'])})

In [12]:
df=pd.DataFrame(table)
df["code_adm"] = pd.to_numeric(df["code_adm"]) #konversi jadi tipe numeric
#tambahan kolom untuk kalkulasi data
df['persen_01_prov'] = (df['pas01'] / (df['pas01'] + df['pas02'])) * 100
df['persen_02_prov'] = (df['pas02'] / (df['pas01'] + df['pas02'])) * 100
df['persen_01_nas'] = (df['pas01'] / df['pas01'].sum()) * 100
df['persen_02_nas'] = (df['pas02'] / df['pas02'].sum()) * 100
df.head()

,code_adm,pas01,pas02,persen_01_prov,persen_02_prov,persen_01_nas,persen_02_nas
0,1,289281,1668557,14.775533,85.224467,0.665235,4.609516
1,6728,2499871,2382888,51.197919,48.802081,5.748743,6.582910
2,12920,282342,1817687,13.444671,86.555329,0.649278,5.021499
3,14086,837442,1388757,37.617571,62.382429,1.925795,3.836547
4,15885,632281,862247,42.306400,57.693600,1.454003,2.382023


In [35]:
indo = 'https://github.com/gwisnu/geojson/raw/master/province.geojson'
m = folium.Map(
    location=[-2, 117],
    tiles='Mapbox bright', #pilihan lain: 'Mapbox bright', ”CartoDB positron”, 'CartoDB dark_matter' ”Stamen Terrain”, “Stamen Toner”, “Stamen Watercolor”
    zoom_start=5
)

#bins = list(df['persen01'].quantile([0, 0.25, 0.5, 0.75, 1]))
#bins = [0, 12.5, 25,37.5, 50,62.5, 75, 87.5, 100]
bins = [0, 2, 5, 10 , 15, 20 ,25, 30 ]

folium.Choropleth(
    geo_data=indo,
    data=df,
    name = 'persen01_all',
    columns=['code_adm', 'persen_01_nas'],
    key_on='feature.properties.code',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.5,
    bins=bins,
    legend_name='Suara Pasangan 01 Nasional (%)',
    reset=True
).add_to(m)
bins = [0, 12.5, 25,37.5, 50,62.5, 75, 87.5, 100]


folium.LayerControl().add_to(m)
m